# Showcase of linear model

## Default settings, including necessary packages and seed

In [8]:
import tensorflow as tf
from load_data import *
from make_base_model import *
from make_feature_model import *
import os
import random as rn
SEED=3
os.environ['PYTHONHASHSEED']=str(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)
rn.seed(SEED)


## Load data

In [ ]:
# load data
X_train, X_test, y_train, y_test, labels = load_data_boston()

## Build model

In [11]:
# build base model
base_model = make_base_model_boston()
base_model.compile(loss='mse', optimizer='adam', metrics=['mae'])

# base_model training
history = base_model.fit(X_train, y_train, epochs=100, batch_size=10, verbose=1, validation_data=(X_test, y_test))


Epoch 1/100
41/41 [==============================] - 0s 4ms/step - loss: 13423.0215 - mae: 111.8327 - val_loss: 9772.0439 - val_mae: 94.4122
Epoch 2/100
41/41 [==============================] - 0s 2ms/step - loss: 8090.6221 - mae: 84.6100 - val_loss: 5786.6436 - val_mae: 69.3039
Epoch 3/100
41/41 [==============================] - 0s 2ms/step - loss: 5427.1753 - mae: 65.3657 - val_loss: 4008.6475 - val_mae: 54.7294
Epoch 4/100
41/41 [==============================] - 0s 2ms/step - loss: 4219.7466 - mae: 54.5239 - val_loss: 3189.3633 - val_mae: 47.6670
Epoch 5/100
41/41 [==============================] - 0s 1ms/step - loss: 3611.2832 - mae: 48.7388 - val_loss: 2733.8799 - val_mae: 43.6684
Epoch 6/100
41/41 [==============================] - 0s 1ms/step - loss: 3220.7468 - mae: 45.1713 - val_loss: 2424.8665 - val_mae: 40.5800
Epoch 7/100
41/41 [==============================] - 0s 1ms/step - loss: 2896.7822 - mae: 42.2472 - val_loss: 2166.1096 - val_mae: 38.0923
Epoch 8/100
41/41 [======

41/41 [==============================] - 0s 1ms/step - loss: 142.5560 - mae: 9.2632 - val_loss: 151.7661 - val_mae: 9.1234
Epoch 61/100
41/41 [==============================] - 0s 1ms/step - loss: 138.4225 - mae: 9.0883 - val_loss: 148.0548 - val_mae: 9.1496
Epoch 62/100
41/41 [==============================] - 0s 1ms/step - loss: 136.0593 - mae: 9.0130 - val_loss: 144.6682 - val_mae: 8.9292
Epoch 63/100
41/41 [==============================] - 0s 2ms/step - loss: 133.5792 - mae: 8.8733 - val_loss: 142.0661 - val_mae: 8.8795
Epoch 64/100
41/41 [==============================] - 0s 1ms/step - loss: 132.1311 - mae: 8.8363 - val_loss: 139.6616 - val_mae: 8.7020
Epoch 65/100
41/41 [==============================] - 0s 1ms/step - loss: 129.1414 - mae: 8.6848 - val_loss: 136.7005 - val_mae: 8.6637
Epoch 66/100
41/41 [==============================] - 0s 1ms/step - loss: 126.3792 - mae: 8.6048 - val_loss: 134.0770 - val_mae: 8.5358
Epoch 67/100
41/41 [==============================] - 0s 1ms/

## Define callback that stops training once the performance acieves the same

In [ ]:
stop_call_back = utilities.EarlyStoppingByLossVal(monitor='val_loss', value=80, verbose=0)


In [ ]:
# uncomment to use the pretrained model
# base_model = tf.keras.models.load_model('./Gold/base_model_{}_no_scaling.h5'.format(name_prefix))

## Set base model untrainable

In [ ]:
base_model.trainable = False

## Define and retrain the feature model

In [15]:
feature_model = make_feature_model(base_model, shape=(len(labels)))

opt = utilities.tf.keras.optimizers.Adam()

feature_model.compile(loss='mse', optimizer=opt, metrics=['mae'], run_eagerly=True)

history = feature_model.fit(X_train, y_train, epochs=100, batch_size=10, verbose=1, validation_data=(X_test, y_test), shuffle=True, callbacks=[stop_call_back])


Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 13)]              0         
                                                                 
 feature_extractor (Sequenti  (None, 13)               13        
 al)                                                             
                                                                 
 sequential_4 (Sequential)   (None, 1)                 14        
                                                                 
Total params: 27
Trainable params: 13
Non-trainable params: 14
_________________________________________________________________
Epoch 1/100
41/41 [==============================] - 0s 10ms/step - loss: 493.7356 - mae: 20.2621 - val_loss: 466.4464 - val_mae: 19.5990
Epoch 2/100
41/41 [==============================] - 0s 9ms/step - loss: 417.3493 - mae: 18.2796 - val_loss: 393.8860 - va

## Print outcomes

In [17]:
weights_importance = np.array(stop_call_back.weights)
weights_importance = np.reshape(arr, (len(arr), np.shape(arr)[-1]))

weights_importance

array([[ 0.07926199,  0.4884857 ,  0.24638276,  0.33488476,  0.39778215,
         0.45051807, -0.41307184, -0.41451162,  0.26689667,  0.38886103,
        -0.44129446,  0.46097392, -0.21057011]], dtype=float32)